# Урок 9. Трансформер
## Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.1 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
     |████████████████████████████████| 86 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Загрузим датасет:

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2022-06-13 22:02:03--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 74.125.196.100, 74.125.196.101, 74.125.196.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.196.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h6pleesbb47ag9vr6ndoiui2ipse6uct/1655157675000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download [following]
--2022-06-13 22:02:07--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h6pleesbb47ag9vr6ndoiui2ipse6uct/1655157675000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-14-c0-docs.googleusercontent.com (doc-14-c0-doc

In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [5]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

Выбрана следующую модель: https://huggingface.co/SkolkovoInstitute/russian_toxicity_classifier

Данная модель предназначена для классификации токсичности текста, является протьюненой моделью Conversational Rubert, обучена на комментариях 2ch.hk, и ok.ru. 

Модель предсказывает 2 класса - toxic and non-toxic. Остальные модели для определения сентимента текста используют 3 класса - positive, neutral and negative.

Использование данной модели:

```
from transformers import BertTokenizer, BertForSequenceClassification

# load tokenizer and model weights
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

# prepare the input
batch = tokenizer.encode('ты супер', return_tensors='pt')

# inference
model(batch)
```

Создадим кастомный датасет:

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels, max_length=10):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=max_length,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [28]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train, 50)
valid_dataset = TwitterDataset(df_val['text'], y_val, 50)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Классифицируем валидационный датасет:

In [30]:
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier').to(device)

model.eval()
total_acc_val = 0.0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask).logits
    # print(output)
                
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
            
print(
    f'Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

Val Accuracy:  0.500


Метрика зависила от максимальной длины твита. Accuracy при разной длине были следующие:

10 - 0.486

20 - 0.492

30 - 0.497

50 - 0.500

При длине твита, выше 50 слов, метрика не менялась.

Учитывая примерное равенство классов, accuracy, равное 50% - это случайное гадание. Попробуем дообучить последний классификационный слой модели.

In [37]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

Последний слой называется classifier. Его и будем подавать в оптимизатор.

In [38]:
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.classifier.parameters(), lr=0.001)  # неполное обучение

In [39]:
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

Parameters full train: 177854978
Parameters transfer learning: 1538


Функция для обучения модели:

In [40]:
def learn_model(epoch_num):
    for epoch_num in range(epoch_num):
        total_acc_train = 0
        total_loss_train = 0

        model.train()
        for train_input, train_label in tqdm(train_loader):
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            train_label = train_label.to(device)

            output = model(input_id, mask).logits
                    
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
                    
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
                
        model.eval()
        total_loss_val, total_acc_val = 0.0, 0.0
        for val_input, val_label in valid_loader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask).logits

            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()
                        
            acc = (output.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
                
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
            | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
            | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
            | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

In [41]:
learn_model(2)

100%|██████████| 2836/2836 [23:12<00:00,  2.04it/s]


Epochs: 1 | Train Loss:  0.009             | Train Accuracy:  0.696             | Val Loss:  0.008             | Val Accuracy:  0.777


100%|██████████| 2836/2836 [23:25<00:00,  2.02it/s]


Epochs: 2 | Train Loss:  0.008             | Train Accuracy:  0.739             | Val Loss:  0.008             | Val Accuracy:  0.781


Модель успешно дообучается, за 2 эпохи метрика на тесте выросла до 78%. Скорее всего можно получить и более высокую метрику, но модель обучается очень долго (46 минут на 2 эпохи, может мне не повезло с выделенным GPU, а может так долго из-за увеличения длины предложения)